## Thurston County Data Clean and Prep

This notebook reads in an MS Access database table downloaded from Thurston County and outputs a .csv file for later use.

The lattitude and longitude are added vai geocoding.

In [ ]:
# Libraries

import pandas as pd
import pandas_access as mdb
import time
from geopy.geocoders import Nominatim

In [ ]:
# Read the apt table from the apt access database

apt_df = mdb.read_table('../data/apt.mdb', "apt")

In [ ]:
apt_df.shape

In [ ]:
apt_df.head()

In [ ]:
apt_df.columns

In [ ]:
# Create a list of the unnecessary columns

dropcolumns = ['PACT_CODE', 'ADDRESS2','COUNTRY', 'SITUS_STRE', 'SITUS_CITY', 'SITUS_ZIP', 'SECTTIE',
       'SD_NAME', 'LEGAL_DESC', 'TOTAL_ACRE','ANNUAL_TAX', 'STATUS_IND', 'PROP_SUBTY', 'O_NEIGHBOR',
       'PROP_TYPE', 'INSPCT_CYC', 'REGION', 'REC_VOLPAG', 'MULT_PARCL',
       'SALE_DATE', 'SALE_PRICE', 'SALE_VRFY', 'CODE2', 'WATER_SRC',
       'SIGMA_YEAR', 'TAXABLE', 'EXEMPT_TY', 'LOCAL_IND', 'TCA', 'CURR_USE',
       'INCORP', 'YEAR_BUILT', 'EFF_YR_BUI', 'RES_QUAL', 'RES_COND',
       'ADDL_STRUC']

In [ ]:
# Write out to a csv file for back-up sake.

# apt_df.to_csv('../data/thurstonraw.csv')

In [ ]:
# read Thurston Raw if the notebook needs to be restarted

thurston_raw_df = pd.read_csv('../data/thurstonraw.csv')

In [ ]:
# Drop unnamed, head
thurston = thurston_raw_df.drop(columns=dropcolumns)
thurston.drop(columns='Unnamed: 0', inplace=True)
thurston.head()

In [ ]:
thurston.dtypes


In [ ]:
# Use only the affected zip codes
# These zip codes were found by combining the zipcode map and the lahar flow map. The affected zipcodes were 
# pulled out manually. This was done to reduce the size the data we were working with. This is not neccessary 
# or advised for the process to run in a production environment, unless we can find a way to programatically d
# determine affected zip codes.

thurston_zips = thurston[(thurston['ZIP'] == '98597') | 
         (thurston['ZIP'] == '98513') | 
         (thurston['ZIP'] == '98516') |
         (thurston['ZIP'] == '98355')]

In [ ]:
# Drop duplicate address

dup_cols = ['ADDRESS1', 'CITY','STATE','ZIP']

thurston_zips.drop_duplicates(subset=dup_cols, inplace=True)

In [ ]:
# Drop nulls from values, as we need the values
thurston_zips.dropna(subset=['BLDG_VALUE','LAND_VALUE','TOTAL_VALU'], inplace=True)

In [ ]:
thurston_zips.shape

In [ ]:
# add lattitude, longitude, and lookup boolean columns

thurston_zips['LAT'] = 0
thurston_zips['LONG'] = 0
thurston_zips['LOOKUP'] = 0

In [ ]:
# reset index
thurston_zips.reset_index(drop=True, inplace=True)

In [ ]:
thurston_zips.head()

In [ ]:
# Set geolocator for lookup

geolocator = Nominatim(user_agent="DSI-SEA-007")

In [ ]:
# Step through the dataframe, looking 

j = 0
lat_list = []
long_list = []

for i, val in thurston_zips.iterrows():
    address = ''

    street = val[1]
    city = val[2]
    state = val[3]
    zip_code = val[4]
               
    address = address + ' ' + street + ' ' + city + ' ' + state + ' ' + zip_code
        
    print(address)
    j += 1
    
    try:
        location = geolocator.geocode(address)
        thurston_zips.loc[i,'LAT'] = location.latitude
        thurston_zips.loc[i,'LONG'] = location.longitude
        thurston_zips.loc[i,'LOOKUP'] = 1
    except:
        lat_list.append(0)
        long_list.append(0)
        thurston_zips.loc[i,'LOOKUP'] = 0
        
    time.sleep(1)
    
    if j == 5:
        break
    
#     location = geolocator.geocode('11231 STEDMAN RD SE OLYMIA WA')
#     print((location.latitude, location.longitude))